In [15]:
import sys
sys.path.append('../Examples')

from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.applications import InceptionResNetV2, VGG19
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dropout, Concatenate, Dense, Average, Dot
from tensorflow.keras.layers import MaxPool2D, Conv2D, Add, ReLU, Lambda
from tensorflow.keras.layers import Input, Flatten, BatchNormalization


from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
# from Utilities import *

import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from time import time

In [2]:
casme = pd.read_csv('./database/CASME/casme.csv')
casme2 = pd.read_csv('./database/CASME2/casme2.csv')
samm = pd.read_csv('./database/SAMM/samm.csv')

In [3]:
def build_SimpleNet(input_shape, output_shape):
    
    input_layer = Input(shape=input_shape, dtype='float64')
    
    conv_layer = Conv2D(16, kernel_size=3, padding='same', activation='relu')(input_layer)
#     max_pool = MaxPool2D(pool_size=2, strides=2)(conv_layer)
    norm = BatchNormalization()(conv_layer)    
    dropout = Dropout(0.5)(norm)
    
    conv_layer = Conv2D(32, kernel_size=3, padding='same', activation='relu')(dropout)
#     max_pool = MaxPool2D(pool_size=2, strides=2)(conv_layer)
    norm = BatchNormalization()(conv_layer)    
    dropout_res_base = Dropout(0.5)(norm)
    
    # residual block branch 1
    conv_layer_rb1 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(dropout_res_base)
    norm_rb1 = BatchNormalization()(conv_layer_rb1)    
    dropout_rb1 = Dropout(0.5)(norm_rb1)
    
    conv_layer_rb1 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(dropout_rb1)
    norm_rb1 = BatchNormalization()(conv_layer_rb1)    
    dropout_rb1 = Dropout(0.5)(norm_rb1)
    
#     res_block_rb1 = Add()([dropout_res_base, dropout_rb1])
#     res_activation_rb1 = ReLU()(res_block_rb1)
    res_activation_rb1 = ReLU()(dropout_rb1)
    
    
    # residual block branch 2
    conv_layer_rb2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(dropout_res_base)
    norm_rb2 = BatchNormalization()(conv_layer_rb2)    
    dropout_rb2 = Dropout(0.5)(norm_rb2)
    
    conv_layer_rb2 = Conv2D(32, kernel_size=3, padding='same', activation='relu')(dropout_rb2)
    norm_rb2 = BatchNormalization()(conv_layer_rb2)    
    dropout_rb2 = Dropout(0.5)(norm_rb2)
    
#     res_block_rb2 = Add()([dropout_res_base, dropout_rb2])
#     res_activation_rb2 = ReLU()(res_block_rb2)
    res_activation_rb2 = ReLU()(dropout_rb2)
    
    # merge res branchses
    average = Average()([res_activation_rb1, norm, res_activation_rb2])
#     norm = BatchNormalization()(average)
    flatten = Flatten()(average)
    dense = Dense(120, activation='relu')(flatten)
    output = Dense(output_shape, activation='softmax')(dense)
    model = Model(inputs=[input_layer], outputs=[output])
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',  #(learning_rate=0.00005)',
               metrics=['accuracy'])
    
    
    return model

In [4]:
def maxPxGrid(frame, gridHeightNum, gridWidthNum):
    (h, w) = frame.shape
    m = int(h / gridHeightNum) + 1 
    n = int(w / gridWidthNum) + 1
    maxGrids = np.zeros([gridHeightNum, gridWidthNum]).astype(int)
    newGrid = np.zeros([m * gridHeightNum, n * gridWidthNum])
    newGrid[: h, : w] = frame
    for i in range(gridHeightNum):
        for j in range(gridWidthNum):
            maxGrids[i][j] = int(newGrid[i * m : (i + 1) * m, j * n : (j + 1) * n].max())
    return maxGrids

In [5]:
def findMove(old_frame, new_frame):
    n = 7
    result = np.zeros([n, n])
    nRange = int((n - 1) / 2)
    r = min([old_frame.shape[0], new_frame.shape[0]])
    c = min([old_frame.shape[1], new_frame.shape[1]])
    for i in range(n):
        i = i - nRange
        for j in range(n):
            j = j - nRange
            if i < 0 and j < 0:
                result[i + nRange][j + nRange] = (new_frame[: r + i, : c + j] - old_frame[-i : r, -j : c]).std().round(3)
            if i < 0 and j >= 0:
                result[i + nRange][j + nRange] = (new_frame[: r + i, j : c] - old_frame[-i : r, : c - j]).std().round(3)
            if i >= 0 and j < 0:
                result[i + nRange][j + nRange] = (new_frame[i : r, : c + j] - old_frame[: r - i, -j : c]).std().round(3)
            if i >= 0 and j >= 0:
                result[i + nRange][j + nRange] = (new_frame[i : r, j : c] - old_frame[: r - i, : c - j]).std().round(3)
    return (int(result.argmin() / n) - nRange, int(result.argmin() % n) - nRange)

In [6]:
emotion_data = pd.read_csv('./database/segmented_parts_colored.csv')

In [7]:
def generateDataset(emotion_data):
    onsetFrame = []
    apexFrame = []
    gridHeightNum = 32
    gridWidthNum = 32
    images = []
    labels = []
    gridDiffTem = np.zeros([gridHeightNum, gridWidthNum]).astype(int)
    r = 0
    c = 0
    for i in range(len(emotion_data)):
        if emotion_data.current[i] == emotion_data.onset[i]:
            onsetFrame = cv2.imread(emotion_data.path[i] + emotion_data.image[i] + str(emotion_data.current[i]) + '.jpg')
            onsetFrame = cv2.cvtColor(onsetFrame, cv2.COLOR_BGR2GRAY)
            onsetFrame = onsetFrame.astype(int)
            r = onsetFrame.shape[0]
            c = onsetFrame.shape[1]
        if emotion_data.current[i] == emotion_data.apex[i]:
            apexFrame = cv2.imread(emotion_data.path[i] + emotion_data.image[i] + str(emotion_data.current[i]) + '.jpg')
            apexFrame = cv2.cvtColor(apexFrame, cv2.COLOR_BGR2GRAY)
            apexFrame = apexFrame.astype(int)
            pxMoved = findMove(onsetFrame, apexFrame)
            xDiff = pxMoved[0]
            yDiff = pxMoved[1]

            if xDiff < 0 and yDiff < 0:
                gridDiffTem = maxPxGrid(apexFrame[: r + xDiff, : c + yDiff], gridHeightNum, gridWidthNum) 
                - maxPxGrid(onsetFrame[-xDiff : r, -yDiff : c], gridHeightNum, gridWidthNum)
            if xDiff < 0 and yDiff >= 0:
                gridDiffTem = maxPxGrid(apexFrame[: r + xDiff, yDiff : c], gridHeightNum, gridWidthNum) 
                - maxPxGrid(onsetFrame[-xDiff : r, : c - yDiff], gridHeightNum, gridWidthNum)
            if xDiff >= 0 and yDiff < 0:
                gridDiffTem = maxPxGrid(apexFrame[xDiff : r, : c + yDiff], gridHeightNum, gridWidthNum) 
                - maxPxGrid(onsetFrame[: r - xDiff, -yDiff : c], gridHeightNum, gridWidthNum)
            if xDiff >= 0 and yDiff >= 0:
                gridDiffTem = maxPxGrid(apexFrame[xDiff : r, yDiff : c], gridHeightNum, gridWidthNum) 
                - maxPxGrid(onsetFrame[: r - xDiff, : c - yDiff], gridHeightNum, gridWidthNum)
            if images == []:
                images = [gridDiffTem]
            else:
                images = images + [gridDiffTem]
            if (emotion_data.emotion[i] == 'happiness' or emotion_data.emotion[i] == 'Happiness'):
                print(emotion_data.emotion[i])
                labels = labels + [1]
            if (emotion_data.emotion[i] == 'fear' or emotion_data.emotion[i] == 'Fear'):
                print(emotion_data.emotion[i])
                labels = labels + [2]
            if (emotion_data.emotion[i] == 'surprise' or emotion_data.emotion[i] == 'Surprise'):
                print(emotion_data.emotion[i])
                labels = labels + [3]
            if (emotion_data.emotion[i] == 'disgust' or emotion_data.emotion[i] == 'Disgust'):
                print(emotion_data.emotion[i])
                labels = labels + [4]
    return images, labels


In [ ]:
images, labels = generateDataset(emotion_data)

In [17]:
np.random.seed(0)
images0 = np.array(images)
perm = np.random.permutation(images0.shape[0])
images0 = images0[perm]
labels0 = np.array(labels)
labels0 = labels0[perm]

In [18]:
images_train, images_test, labels_train, labels_test = train_test_split(images0, labels0, test_size = 0.2, random_state = 0)

In [23]:
model = Sequential()
model.add(Input(shape = (32, 32)))
model.add(Flatten())
model.add(Dense(5,activation = 'softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 5125      
Total params: 5,125
Trainable params: 5,125
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = Adam(learning_rate = 0.001),
             metrics = ['accuracy'])
checkpt = ModelCheckpoint(filepath="simplest_network.h5",
                              verbose=2,
                              save_best_only=True,
                              monitor='val_accuracy')

In [25]:
start_time = time()
hist = model.fit(images_train, labels_train,
                      validation_split = 0.2,
                      epochs = 1000,
                      shuffle = True,
                      callbacks = [checkpt])
end_time = time()

Train on 161 samples, validate on 41 samples
Epoch 1/1000
 32/161 [====>.........................] - ETA: 0s - loss: 49.1630 - accuracy: 0.6250
Epoch 00001: val_accuracy improved from -inf to 0.48780, saving model to simplest_network.h5
161/161 [==============================] - 0s 2ms/sample - loss: 80.9765 - accuracy: 0.3665 - val_loss: 65.9017 - val_accuracy: 0.4878
Epoch 2/1000
 32/161 [====>.........................] - ETA: 0s - loss: 74.7198 - accuracy: 0.4375
Epoch 00002: val_accuracy did not improve from 0.48780
161/161 [==============================] - 0s 160us/sample - loss: 77.3560 - accuracy: 0.3292 - val_loss: 73.7891 - val_accuracy: 0.4634
Epoch 3/1000
 32/161 [====>.........................] - ETA: 0s - loss: 41.8317 - accuracy: 0.5938
Epoch 00003: val_accuracy did not improve from 0.48780
161/161 [==============================] - 0s 91us/sample - loss: 74.9839 - accuracy: 0.4845 - val_loss: 108.9446 - val_accuracy: 0.2683
Epoch 4/1000
 32/161 [====>...................

Epoch 29/1000
 32/161 [====>.........................] - ETA: 0s - loss: 22.9180 - accuracy: 0.5625
Epoch 00029: val_accuracy did not improve from 0.51220
161/161 [==============================] - 0s 121us/sample - loss: 25.1430 - accuracy: 0.5776 - val_loss: 35.1838 - val_accuracy: 0.3659
Epoch 30/1000
 32/161 [====>.........................] - ETA: 0s - loss: 14.6758 - accuracy: 0.6250
Epoch 00030: val_accuracy did not improve from 0.51220
161/161 [==============================] - 0s 116us/sample - loss: 24.0529 - accuracy: 0.4969 - val_loss: 39.4144 - val_accuracy: 0.4634
Epoch 31/1000
 32/161 [====>.........................] - ETA: 0s - loss: 15.1918 - accuracy: 0.6875
Epoch 00031: val_accuracy did not improve from 0.51220
161/161 [==============================] - 0s 122us/sample - loss: 17.3818 - accuracy: 0.6273 - val_loss: 73.9705 - val_accuracy: 0.2195
Epoch 32/1000
 32/161 [====>.........................] - ETA: 0s - loss: 44.1506 - accuracy: 0.3750
Epoch 00032: val_accurac

 32/161 [====>.........................] - ETA: 0s - loss: 33.0243 - accuracy: 0.6875
Epoch 00057: val_accuracy did not improve from 0.51220
161/161 [==============================] - 0s 124us/sample - loss: 16.5800 - accuracy: 0.7205 - val_loss: 146.6817 - val_accuracy: 0.3171
Epoch 58/1000
 32/161 [====>.........................] - ETA: 0s - loss: 21.3946 - accuracy: 0.4688
Epoch 00058: val_accuracy did not improve from 0.51220
161/161 [==============================] - 0s 128us/sample - loss: 25.8486 - accuracy: 0.5776 - val_loss: 135.8601 - val_accuracy: 0.4878
Epoch 59/1000
 32/161 [====>.........................] - ETA: 0s - loss: 52.7949 - accuracy: 0.6250
Epoch 00059: val_accuracy did not improve from 0.51220
161/161 [==============================] - 0s 125us/sample - loss: 23.9592 - accuracy: 0.6646 - val_loss: 146.7299 - val_accuracy: 0.2683
Epoch 60/1000
 32/161 [====>.........................] - ETA: 0s - loss: 11.9720 - accuracy: 0.5312
Epoch 00060: val_accuracy did not i

 32/161 [====>.........................] - ETA: 0s - loss: 10.4094 - accuracy: 0.6562
Epoch 00085: val_accuracy did not improve from 0.53659
161/161 [==============================] - 0s 124us/sample - loss: 8.6372 - accuracy: 0.7391 - val_loss: 62.4700 - val_accuracy: 0.4634
Epoch 86/1000
 32/161 [====>.........................] - ETA: 0s - loss: 8.1128 - accuracy: 0.7500
Epoch 00086: val_accuracy did not improve from 0.53659
161/161 [==============================] - 0s 124us/sample - loss: 6.4147 - accuracy: 0.7578 - val_loss: 56.7820 - val_accuracy: 0.4390
Epoch 87/1000
 32/161 [====>.........................] - ETA: 0s - loss: 14.2305 - accuracy: 0.7812
Epoch 00087: val_accuracy did not improve from 0.53659
161/161 [==============================] - 0s 80us/sample - loss: 6.7079 - accuracy: 0.7702 - val_loss: 54.4039 - val_accuracy: 0.4634
Epoch 88/1000
 32/161 [====>.........................] - ETA: 0s - loss: 6.4513 - accuracy: 0.7500
Epoch 00088: val_accuracy did not improve fr

 32/161 [====>.........................] - ETA: 0s - loss: 21.7550 - accuracy: 0.5000
Epoch 00113: val_accuracy did not improve from 0.56098
161/161 [==============================] - 0s 121us/sample - loss: 10.2104 - accuracy: 0.6894 - val_loss: 82.4834 - val_accuracy: 0.3171
Epoch 114/1000
 32/161 [====>.........................] - ETA: 0s - loss: 14.4257 - accuracy: 0.5625
Epoch 00114: val_accuracy did not improve from 0.56098
161/161 [==============================] - 0s 124us/sample - loss: 10.9174 - accuracy: 0.6957 - val_loss: 71.3625 - val_accuracy: 0.3659
Epoch 115/1000
 32/161 [====>.........................] - ETA: 0s - loss: 12.5648 - accuracy: 0.7188
Epoch 00115: val_accuracy did not improve from 0.56098
161/161 [==============================] - 0s 126us/sample - loss: 19.8340 - accuracy: 0.5963 - val_loss: 75.5894 - val_accuracy: 0.4390
Epoch 116/1000
 32/161 [====>.........................] - ETA: 0s - loss: 25.7474 - accuracy: 0.6250
Epoch 00116: val_accuracy did not i

...

Epoch 889/1000
 32/161 [====>.........................] - ETA: 0s - loss: 8.7054e-06 - accuracy: 1.0000
Epoch 00889: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 98us/sample - loss: 6.3961e-06 - accuracy: 1.0000 - val_loss: 55.4184 - val_accuracy: 0.5610
Epoch 890/1000
 32/161 [====>.........................] - ETA: 0s - loss: 6.9621e-06 - accuracy: 1.0000
Epoch 00890: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 129us/sample - loss: 6.3850e-06 - accuracy: 1.0000 - val_loss: 55.4177 - val_accuracy: 0.5610
Epoch 891/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.2464e-05 - accuracy: 1.0000
Epoch 00891: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 90us/sample - loss: 6.3739e-06 - accuracy: 1.0000 - val_loss: 55.4156 - val_accuracy: 0.5610
Epoch 892/1000
 32/161 [====>.........................] - ETA: 0s - loss: 2.3469e-07 - accuracy: 1.0000
E

Epoch 00916: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 124us/sample - loss: 6.1563e-06 - accuracy: 1.0000 - val_loss: 55.4161 - val_accuracy: 0.5610
Epoch 917/1000
 32/161 [====>.........................] - ETA: 0s - loss: 5.5169e-06 - accuracy: 1.0000
Epoch 00917: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 124us/sample - loss: 6.1452e-06 - accuracy: 1.0000 - val_loss: 55.4166 - val_accuracy: 0.5610
Epoch 918/1000
 32/161 [====>.........................] - ETA: 0s - loss: 3.3154e-06 - accuracy: 1.0000
Epoch 00918: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 87us/sample - loss: 6.1333e-06 - accuracy: 1.0000 - val_loss: 55.4166 - val_accuracy: 0.5610
Epoch 919/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.5720e-06 - accuracy: 1.0000
Epoch 00919: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 97u

Epoch 944/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.0158e-05 - accuracy: 1.0000
Epoch 00944: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 124us/sample - loss: 5.8868e-06 - accuracy: 1.0000 - val_loss: 55.4241 - val_accuracy: 0.5610
Epoch 945/1000
 32/161 [====>.........................] - ETA: 0s - loss: 5.9713e-06 - accuracy: 1.0000
Epoch 00945: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 99us/sample - loss: 5.9053e-06 - accuracy: 1.0000 - val_loss: 55.4461 - val_accuracy: 0.5610
Epoch 946/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.2631e-05 - accuracy: 1.0000
Epoch 00946: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 106us/sample - loss: 5.8845e-06 - accuracy: 1.0000 - val_loss: 55.4570 - val_accuracy: 0.5610
Epoch 947/1000
 32/161 [====>.........................] - ETA: 0s - loss: 3.3265e-06 - accuracy: 1.0000


Epoch 972/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.3597e-06 - accuracy: 1.0000
Epoch 00972: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 132us/sample - loss: 5.5795e-06 - accuracy: 1.0000 - val_loss: 55.4307 - val_accuracy: 0.5610
Epoch 973/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.3317e-05 - accuracy: 1.0000
Epoch 00973: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 82us/sample - loss: 5.5951e-06 - accuracy: 1.0000 - val_loss: 55.4234 - val_accuracy: 0.5610
Epoch 974/1000
 32/161 [====>.........................] - ETA: 0s - loss: 2.0862e-07 - accuracy: 1.0000
Epoch 00974: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 83us/sample - loss: 5.5662e-06 - accuracy: 1.0000 - val_loss: 55.4250 - val_accuracy: 0.5610
Epoch 975/1000
 32/161 [====>.........................] - ETA: 0s - loss: 1.2182e-06 - accuracy: 1.0000
E

Epoch 00999: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 135us/sample - loss: 5.3130e-06 - accuracy: 1.0000 - val_loss: 55.4372 - val_accuracy: 0.5610
Epoch 1000/1000
 32/161 [====>.........................] - ETA: 0s - loss: 7.0035e-07 - accuracy: 1.0000
Epoch 01000: val_accuracy did not improve from 0.58537
161/161 [==============================] - 0s 124us/sample - loss: 5.3012e-06 - accuracy: 1.0000 - val_loss: 55.4376 - val_accuracy: 0.5610


In [26]:
#for simplest_network
model_best = load_model('simplest_network.h5')
loss_train,acc_train = model_best.evaluate(images_train,labels_train,verbose=0)
loss_test,acc_test = model_best.evaluate(images_test,labels_test,verbose=0)
print()
print('train loss',loss_train.round(3))
print(' test loss',loss_test.round(3))
print()
print('train error',(1 - acc_train).round(2))
print(' test error',(1 - acc_test).round(2))
print()
print('time', (end_time - start_time) / 60)


train loss 10.726
 test loss 73.776

train error 0.13
 test error 0.47

time 0.3404859145482381


In [51]:
model = build_SimpleNet(input_shape=(32, 32, 1), output_shape=5)

model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 32, 32, 16)   32          input_10[0][0]                

In [82]:
images_train.resize(np.append(images_train.shape, [1]))
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = Adam(learning_rate = 0.001),
             metrics = ['accuracy'])

checkpt = ModelCheckpoint(filepath="simplest_network.h5",
                              verbose=2,
                              save_best_only=True,
                              monitor='val_accuracy')

model_log = model.fit(images_train, labels_train,
                      validation_split = 0.2,
                      epochs = 50,
                      shuffle = True,
                      callbacks = [checkpt])

Train on 161 samples, validate on 41 samples
Epoch 1/50
160/161 [============================>.] - ETA: 0s - loss: 0.4837 - accuracy: 0.8875
Epoch 00001: val_accuracy improved from -inf to 0.43902, saving model to simplest_network.h5
161/161 [==============================] - 2s 13ms/sample - loss: 0.4807 - accuracy: 0.8882 - val_loss: 4.6593 - val_accuracy: 0.4390
Epoch 2/50
160/161 [============================>.] - ETA: 0s - loss: 0.4512 - accuracy: 0.8625
Epoch 00002: val_accuracy did not improve from 0.43902
161/161 [==============================] - 1s 6ms/sample - loss: 0.5044 - accuracy: 0.8571 - val_loss: 4.6036 - val_accuracy: 0.3902
Epoch 3/50
160/161 [============================>.] - ETA: 0s - loss: 0.3358 - accuracy: 0.8750
Epoch 00003: val_accuracy improved from 0.43902 to 0.58537, saving model to simplest_network.h5
161/161 [==============================] - 1s 6ms/sample - loss: 0.3337 - accuracy: 0.8758 - val_loss: 1.7776 - val_accuracy: 0.5854
Epoch 4/50
160/161 [===

160/161 [============================>.] - ETA: 0s - loss: 0.0857 - accuracy: 0.9625
Epoch 00029: val_accuracy did not improve from 0.75610
161/161 [==============================] - 1s 5ms/sample - loss: 0.0899 - accuracy: 0.9565 - val_loss: 2.2391 - val_accuracy: 0.5366
Epoch 30/50
160/161 [============================>.] - ETA: 0s - loss: 0.0973 - accuracy: 0.9625
Epoch 00030: val_accuracy did not improve from 0.75610
161/161 [==============================] - 1s 5ms/sample - loss: 0.0968 - accuracy: 0.9627 - val_loss: 2.0618 - val_accuracy: 0.5610
Epoch 31/50
160/161 [============================>.] - ETA: 0s - loss: 0.0781 - accuracy: 0.9812
Epoch 00031: val_accuracy did not improve from 0.75610
161/161 [==============================] - 1s 6ms/sample - loss: 0.0776 - accuracy: 0.9814 - val_loss: 2.3869 - val_accuracy: 0.5122
Epoch 32/50
160/161 [============================>.] - ETA: 0s - loss: 0.0830 - accuracy: 0.9688
Epoch 00032: val_accuracy did not improve from 0.75610
161/1

In [83]:
#for highly overfitting network
images_test.resize(np.append(images_test.shape, [1]))

model_best = load_model('simplest_network.h5')
loss_train,acc_train = model_best.evaluate(images_train,labels_train,verbose=0)
loss_test,acc_test = model_best.evaluate(images_test,labels_test,verbose=0)
print()
print('train loss',loss_train.round(3))
print(' test loss',loss_test.round(3))
print()
print('train error',(1 - acc_train).round(2))
print(' test error',(1 - acc_test).round(2))


train loss 0.947
 test loss 0.618

train error 0.27
 test error 0.22
